# Autograd: 자동 미분

## Autograd Package

- Autograd 패키지는 tensor의 모든 연산에 자동 미분을 제공합니다. 이는 define-by-run의 프레임워크로 코드를 어떻게 작성하느냐에 따라 역전파가 정의된다는 뜻입니다. 역전파는 학습과정의 매 단계마다 달라집니다.

- .requires_grad 속성을 True로 설정하면 해당 tensor의 모든 연산을 추적합니다. 계산이 완료된 후 .backward()를 호출해 gradient를 자동으로 계산할 수 있습니다. 이 tensor의 gradient는 .grad에 누적됩니다.

- 연산 기록을 추적하는 것을 멈추기 위해 코드 블럭을 with torch.no_grad():로 감쌀 수 있습니다. gradient는 필요 없지만 requires_grad=True가 설정되어 학습 가능한 Parameter(매개변수)를 갖는 모델을 평가할 때 유용합니다.

In [1]:
import torch

# 예시 1

In [2]:
a = torch.randn(2,2)
a = ((a*3)/(a-1)) 
print(a.requires_grad)
print(a.grad_fn) # 사용자가 만든 텐서의 grad_fn은 none입니다.

False
None


In [3]:
a

tensor([[ 1.8504,  8.2137],
        [-0.3290,  1.4948]])

In [4]:
a.requires_grad_(True)
print(a.requires_grad)
print(a.grad_fn)

True
None


In [5]:
a

tensor([[ 1.8504,  8.2137],
        [-0.3290,  1.4948]], requires_grad=True)

In [6]:
b = (a*a).sum()
print(b.grad_fn) #requires_grad_(True)로 지정하고 연산하면 이렇게 grad_fn가 생깁니다.

In [7]:
b

tensor(73.2324, grad_fn=<SumBackward0>)

# 예시 2

In [8]:
x = torch.ones(2,2,requires_grad=True) #tensor를 생성하고 requires_grad=True로 연산을 기록합니다.
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [9]:
y = x+2 #gradient function이 자동으로 포함됩니다.
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [10]:
z = y*y*3
out = z.mean()
print(z,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


![](./figures/AUTO1)

![](./figures2/auto1.PNG)

![대체 텍스트](figures/auto1.png)

### Gradient

![대체 텍스트](figures/auto2.PNG)

- 직접 계산

In [11]:
x_grad = 1.5 * (x+2)
x_grad

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]], grad_fn=<MulBackward0>)

- Autograd 이용 

In [12]:
print(out) # out = 3(x+2)*2
out.backward()

tensor(27., grad_fn=<MeanBackward0>)


In [13]:
print(x)
print(x.grad) # d(out)/dx 를 출력합니다.

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


# 예시 3

In [14]:
x = torch.randn(3,requires_grad=True)

y=x*2
i=1

while y.data.norm() < 1000:
    i += 1
    y = y*2
    
    print(y,y.data.norm())
    print(i)

tensor([ 1.4419, -0.4544, -6.8348], grad_fn=<MulBackward0>) tensor(7.0000)
2
tensor([  2.8837,  -0.9088, -13.6696], grad_fn=<MulBackward0>) tensor(14.0000)
3
tensor([  5.7675,  -1.8176, -27.3391], grad_fn=<MulBackward0>) tensor(27.9999)
4
tensor([ 11.5350,  -3.6351, -54.6783], grad_fn=<MulBackward0>) tensor(55.9999)
5
tensor([  23.0700,   -7.2703, -109.3566], grad_fn=<MulBackward0>) tensor(111.9998)
6
tensor([  46.1400,  -14.5406, -218.7132], grad_fn=<MulBackward0>) tensor(223.9995)
7
tensor([  92.2799,  -29.0812, -437.4264], grad_fn=<MulBackward0>) tensor(447.9990)
8
tensor([ 184.5599,  -58.1624, -874.8527], grad_fn=<MulBackward0>) tensor(895.9980)
9
tensor([  369.1198,  -116.3247, -1749.7054], grad_fn=<MulBackward0>) tensor(1791.9961)
10


In [15]:
print(y)
gradients = torch.tensor([0.1,1.0,0.0001],dtype=torch.float)
y.backward(gradients)
print(x.grad) # d(y)/d(x) 를 출력합니다

tensor([  369.1198,  -116.3247, -1749.7054], grad_fn=<MulBackward0>)
tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


# 예시 4

In [16]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad) #tensor들의 연산 기록 추적을 막을 수 있습니다.
     
print((x**2).requires_grad)

True
True
False
True


- autograd package에 대한 더 자세한 정보는 다음의 링크를 참고하세요. https://pytorch.org/docs/stable/torch.html